In [2]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
import numpy as np
import cv2,os,time,random,zipfile,fitz,re,shutil
import pandas as pd


In [28]:
#读取数据，筛选出需要预测的数据
my_Data1="C:/Users/ASUS/Desktop/problem c/mydata.xlsx"
mydata=pd.read_excel(my_Data1,engine='openpyxl')#用openpyxl代替xlrd，不然不支持xlsx文件

# 把两个表格中的数据融合在一起
da=mydata[['GlobalID','Lab Status','FileName']]#在数据中选取感兴趣的数据
DATA=[]

length=len(da['GlobalID'])
for i in range(length):
    temp=da.loc[i,'Lab Status']
    if temp=='Positive ID' or temp=='Unprocessed' or temp=='Unverified':#如果是预测
        if da.loc[i,'FileName'] :
            temp=[]
            temp.append(da.loc[i,'GlobalID'])
            temp.append(da.loc[i,'Lab Status'])
            temp.append(da.loc[i,'FileName'])
            DATA.append(temp)
# 增加图片文件信息
pd_DATA=pd.DataFrame(DATA,columns=['GlobalID','Lab Status','FileName'])



In [10]:
# 进行预测
model=load_model("C:/Users/ASUS/Desktop/problem c/model03_59_0.02.h5")
IMG_SIZE = 64

In [29]:
path="C:/Users/ASUS/Desktop/problem c/2021MCM_ProblemC_Files"

Positive_ID_Probability=[]
for FileName in  pd_DATA['FileName'].values.tolist():
    this_path=os.path.join(path,FileName)
    if ('png' in FileName or 'jpg'in FileName or 'jfif'in FileName):
        create_data_from_img(this_path,Positive_ID_Probability)
    elif ('pdf'in FileName):
        create_data_from_pdf(this_path,Positive_ID_Probability)
    elif('docx'in FileName):
        create_data_from_docx(this_path,Positive_ID_Probability)
        
    elif('MOV'in FileName or 'mp4'in FileName):
        create_data_from_video(this_path,Positive_ID_Probability)
    elif('zip'in FileName):
        create_data_from_zip(this_path,Positive_ID_Probability)
    else:
        Positive_ID_Probability.append("Error!")

ipykernel_launcher:3: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
run:0.29863799999998264s
Total take 1 imges
run:0.650297900000055s
Total take 2 imges
run:0.2796614999999747s
Total take 1 imges


In [32]:
pd_DATA['Positive ID Probability']=Positive_ID_Probability
# 把处理好的数据保存下来
writer=pd.ExcelWriter('C:/Users/ASUS/Desktop/problem c/mydata4.xlsx')
pd_DATA.to_excel(writer,index=False)
writer.save()

In [25]:
def predict_from_img(this_path,img=[]):
    #用图片数据预测Positive_ID_Probability的概率
    if img==[]:
        img_array= cv2.imread(this_path,1) #读取照片
    else:
        img_array=img
    image = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  
    image = image.astype("float") / 255.0
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)
    prob = model.predict(image)[0]

    result=prob[0]*100
    return result

def create_data_from_img(this_path,Positive_ID_Probability):
    #如果数据是图片，png、jpg、jfif
    Positive_ID_Probability.append(predict_from_img(this_path))
    
    
def create_data_from_pdf(this_path,Positive_ID_Probability):
    # 在PDF中提取照片信息
    pic_path = this_path[0:len(this_path)-4] #解压后的路径
    # 创建保存图片的文件夹，如果已经存在则不创建
    if os.path.exists(pic_path):
        pass
    else:
        os.mkdir(pic_path)

    t0 = time.clock()                          # 生成图片初始时间
    checkXO = r"/Type(?= */XObject)"           # 使用正则表达式来查找图片
    checkIM = r"/Subtype(?= */Image)"
    doc = fitz.open(this_path)                      # 打开pdf文件
    imgcount = 0                               # 图片计数
    lenXREF = doc._getXrefLength()             # 获取对象数量长度
     
    result=0

    # 遍历每一个对象
    for i in range(1, lenXREF):
        text = doc._getXrefString(i)            # 定义对象字符串
        isXObject = re.search(checkXO, text)    # 使用正则表达式查看是否是对象
        isImage = re.search(checkIM, text)      # 使用正则表达式查看是否是图片
        if not isXObject or not isImage:        # 如果不是对象也不是图片，则continue
            continue
        imgcount += 1

        pix = fitz.Pixmap(doc, i)               # 生成图像对象
        new_name = "Img{}.png".format(imgcount) # 生成图片的名称
        if pix.n < 5:                           # 如果pix.n<5,可以直接存为PNG
            pix.writePNG(os.path.join(pic_path, new_name))
        else:                                   # 否则先转换CMYK
            pix0 = fitz.Pixmap(fitz.csRGB, pix)
            pix0.writePNG(os.path.join(pic_path, new_name))
            pix0 = None
        pix = None                              # 释放资源
        t1 = time.clock()                       # 图片完成时间
        print("run:{}s".format(t1 - t0))
        print("Total take {} imges".format(imgcount))

        # 进到文件夹里边读取图像数据
        result=(result+predict_from_img(os.path.join(pic_path, new_name)))/2#取平均值
    #记录预测的结果
    Positive_ID_Probability.append(result)

def create_data_from_docx(this_path,Positive_ID_Probability):
    
    path=this_path
    zip_path=this_path[0:len(this_path)-4]+'zip'
    tmp_path=this_path[0:len(this_path)-5]+'temp'
    store_path=this_path[0:len(this_path)-5]

    # 创建保存图片的文件夹，如果已经存在则不创建
    if os.path.exists(tmp_path):
        pass
    else:
        os.mkdir(tmp_path)
    if os.path.exists(store_path):
        pass
    else:
        os.mkdir(store_path)
    
    # 将docx文件重命名为zip文件
    os.rename(path, zip_path)
    f = zipfile.ZipFile(zip_path, 'r')# 进行解压
    for file in f.namelist():# 将图片提取并保存
        f.extract(file, tmp_path)
    f.close()# 释放该zip文件

    # 将docx文件从zip还原为docx
    os.rename(zip_path, path)
    # 得到缓存文件夹中图片列表
    pic = os.listdir(os.path.join(tmp_path, 'word/media'))

    result=0

    # 将图片复制到最终的文件夹中
    for i in pic:
        # 根据word的路径生成图片的名称
        new_name = path.replace('\\', '_')
        new_name = new_name.replace(':', '') + '_' + i
        shutil.copy(os.path.join(tmp_path + '/word/media', i), os.path.join(store_path, new_name))

        result=(result+predict_from_img(os.path.join(store_path, new_name)))/2#取平均值
    #记录预测的结果
    Positive_ID_Probability.append(result)

    # 删除缓冲文件夹中的文件，用以存储下一次的文件
    for i in os.listdir(tmp_path):
        # 如果是文件夹则删除
        if os.path.isdir(os.path.join(tmp_path, i)):
            shutil.rmtree(os.path.join(tmp_path, i))


def create_data_from_video(this_path,Positive_ID_Probability):
    # 从视频中读取数据，一帧一帧读取
    cap = cv2.VideoCapture(this_path)#打开视频
   
    result=0
    
    while cap.isOpened():#如果成功打开
        rval, image = cap.read()
        if rval==True:
            result =(result+predict_from_img(this_path,image))/2
        else:
            break
    Positive_ID_Probability.append(result)
    
def create_data_from_zip(this_path,Positive_ID_Probability):
    #从压缩包中得到数据
    result=0
    with zipfile.ZipFile(this_path, mode='r') as zfile: # 只读方式打开压缩包
 
        for name in zfile.namelist():  # 获取zip文档内所有文件的名称列表
            if ('.jpg' not in name) or ('.JPG'not in name):# 仅读取.jpg图片，过滤掉文件夹，及其他非.jpg后缀文件
                continue
            #print(name)
            with zfile.open(name,mode='r') as image_file:
                content = image_file.read() # 一次性读入整张图片信息
                image = np.asarray(bytearray(content), dtype='uint8')
                result=(result+predict_from_img(this_path,image))/2
    zfile.close()
    Positive_ID_Probability.append(result)